In [1]:
import os
import sys
import subprocess
import time
import traci

# Dossier de configuration
SUMO_FOLDER = "sumo_configs"
if not os.path.exists(SUMO_FOLDER):
    os.makedirs(SUMO_FOLDER)

# Configuration du chemin SUMO (Ajustez si nécessaire)
if "SUMO_HOME" not in os.environ:
    os.environ["SUMO_HOME"] = r"C:\Program Files (x86)\Eclipse\Sumo"

sumo_bin = os.path.join(os.environ["SUMO_HOME"], "bin")
if sumo_bin not in os.environ["PATH"]:
    os.environ["PATH"] += os.pathsep + sumo_bin

sys.path.append(os.path.join(os.environ["SUMO_HOME"], "tools"))
print(f"✅ Environnement prêt. SUMO_HOME : {os.environ['SUMO_HOME']}")

✅ Environnement prêt. SUMO_HOME : C:\Program Files (x86)\Eclipse\Sumo\


In [15]:
# 1. Noeuds
nodes_xml = """<nodes>
    <node id="C" x="0" y="0" type="traffic_light"/>
    <node id="N" x="0" y="100"/>
    <node id="S" x="0" y="-100"/>
    <node id="E" x="100" y="0"/>
    <node id="W" x="-100" y="0"/>
</nodes>"""

# 2. Axes avec Trottoirs (sidewalkWidth)
edges_xml = """<edges>
    <edge id="N2C" from="N" to="C" priority="1" numLanes="2" speed="13.89" sidewalkWidth="2.0"/>
    <edge id="C2S" from="C" to="S" priority="1" numLanes="2" speed="13.89" sidewalkWidth="2.0"/>
    <edge id="E2C" from="E" to="C" priority="1" numLanes="2" speed="13.89" sidewalkWidth="2.0"/>
    <edge id="C2W" from="C" to="W" priority="1" numLanes="2" speed="13.89" sidewalkWidth="2.0"/>
</edges>"""

# 3. Routes, Véhicules et Piétons
routes_xml = """<routes>
    <vType id="car" accel="1.0" decel="4.5" sigma="0.5" length="5" minGap="2.5" maxSpeed="13.89" guiShape="passenger"/>
    <vType id="emergency" accel="2.5" decel="5.0" sigma="0.5" length="5" minGap="2.5" maxSpeed="18.0" guiShape="emergency" color="red"/>
    <vType id="pedestrian" vClass="pedestrian" 
           width="2.0" length="1.5" height="2.5" 
           color="yellow" guiShape="person"/>

    <flow id="fN" type="car" begin="0" end="3600" probability="0.1" from="N2C" to="C2S"/>
    <flow id="fE" type="car" begin="0" end="3600" probability="0.1" from="E2C" to="C2W"/>
    
    <personFlow id="p1" begin="0" end="3600" probability="0.2">
        <walk from="N2C" to="C2S"/>
    </personFlow>

    <vehicle id="amb_1" type="emergency" depart="50" departLane="1">
        <route edges="N2C C2S"/>
    </vehicle>
</routes>"""

# 4. Configuration SUMO
config_xml = """<configuration>
    <input>
        <net-file value="network.net.xml"/>
        <route-files value="routes.rou.xml"/>
    </input>
    <time><begin value="0"/><end value="1000"/></time>
</configuration>"""

# Sauvegarde des fichiers
files = {"nodes.nod.xml": nodes_xml, "edges.edg.xml": edges_xml, 
         "routes.rou.xml": routes_xml, "config.sumocfg": config_xml}

for name, content in files.items():
    with open(os.path.join(SUMO_FOLDER, name), "w") as f:
        f.write(content)

# Compilation du réseau
# Compilation du réseau avec détection automatique des passages piétons
# Compilation du réseau (à mettre dans votre cellule de génération)
subprocess.run(["netconvert", 
                "--node-files", os.path.join(SUMO_FOLDER, "nodes.nod.xml"),
                "--edge-files", os.path.join(SUMO_FOLDER, "edges.edg.xml"),
                "--crossings.guess", "true", 
                "-o", os.path.join(SUMO_FOLDER, "network.net.xml")], shell=True)
print("✅ Réseau compilé avec succès.")

✅ Réseau compilé avec succès.


In [16]:
import traci
import sys
import time

try:
    traci.start([sumo_binary, "-c", config_path, "--start"])
    TL_ID = "C"
    step = 0
    
    while step < 1000:
        traci.simulationStep()
        
        # 1. RÉCUPÉRATION DES DONNÉES
        qN = traci.edge.getLastStepHaltingNumber("N2C")
        qE = traci.edge.getLastStepHaltingNumber("E2C")
        
        # Détection des piétons (sur les axes ou passages)
        all_persons = traci.person.getIDList()
        nb_pietons = 0
        for p_id in all_persons:
            pos = traci.person.getRoadID(p_id)
            if pos in ["N2C", "E2C", "C2S", "C2W"] or pos.startswith(":"):
                nb_pietons += 1
        
        # Détection des urgences (Ambulances)
        emergency_N = any(traci.vehicle.getTypeID(v) == "emergency" for v in traci.edge.getLastStepVehicleIDs("N2C"))
        emergency_E = any(traci.vehicle.getTypeID(v) == "emergency" for v in traci.edge.getLastStepVehicleIDs("E2C"))

        current_phase = traci.trafficlight.getPhase(TL_ID)

        # 2. LOGIQUE DE DÉCISION PAR PRIORITÉ
        
        # --- RÈGLE 1 : PRIORITÉ MAXIMALE AUX URGENCES ---
        if emergency_N:
            traci.trafficlight.setPhase(TL_ID, 0) # Vert Nord (Axe de l'ambulance)
            status = "🚨 URGENCE NORD (PRIORITÉ 1)"
        elif emergency_E:
            traci.trafficlight.setPhase(TL_ID, 2) # Vert Est (Axe de l'ambulance)
            status = "🚨 URGENCE EST (PRIORITÉ 1)"
            
        # --- RÈGLE 2 : PRIORITÉ AUX PIÉTONS (Si pas d'urgence) ---
        elif nb_pietons > 0:
            # Ici, on peut soit laisser le feu actuel, soit passer à une phase sécurisée
            status = "🚶 PIÉTONS DÉTECTÉS (PRIORITÉ 2)"
            # Note : On ne change pas brusquement pour ne pas écraser le piéton
            
        # --- RÈGLE 3 : GESTION DU TRAFIC NORMAL (Densité) ---
        else:
            status = "Trafic Normal (Priorité 3)"
            if qN > qE + 2 and current_phase == 2:
                traci.trafficlight.setPhase(TL_ID, 0)
            elif qE > qN + 2 and current_phase == 0:
                traci.trafficlight.setPhase(TL_ID, 2)

        # Monitoring
        sys.stdout.write(f"\rPas: {step} | Urgence? {'OUI' if (emergency_N or emergency_E) else 'NON'} | Piétons: {nb_pietons} | État: {status}      ")
        sys.stdout.flush()
        
        step += 1
        time.sleep(0.05)

    traci.close()
except Exception as e:
    print(f"\n❌ Erreur : {e}")
    try: traci.close()
    except: pass

Pas: 999 | Urgence? NON | Piétons: 35 | État: 🚶 PIÉTONS DÉTECTÉS (PRIORITÉ 2)      